In [74]:
from google.colab import drive
drive.mount('/content/data')

Drive already mounted at /content/data; to attempt to forcibly remount, call drive.mount("/content/data", force_remount=True).


In [75]:
import easydict
args = easydict.EasyDict()

# path 정보
args.default_path = '/content/data/MyDrive/팀프로젝트/'
args.train_csv = args.default_path+'train.csv'
args.test_csv = args.default_path+'test.csv'
args.default_submission_csv = args.default_path+'submission.csv'

args.submission_csv = 'submission_250814.csv'#제출용파일
args.save_results = "model_results.json"

# 데이터 분석을 위한 변수들
args.random_state = 21
args.results = []

# 라이브러리 임포트

In [76]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score
import json
import easydict
import matplotlib.pyplot as plt
import seaborn as sns

# 설정 및 파일 경로

In [77]:
args = easydict.EasyDict()

# path 정보
args.default_path = '/content/data/MyDrive/팀프로젝트/'
args.train_csv = args.default_path+'train.csv'
args.test_csv = args.default_path+'test.csv'
args.default_submission_csv = args.default_path+'submission.csv'

# 결과 저장을 위한 파일 이름
args.submission_csv = 'submission_teamp3.csv' # 제출용 파일 이름 지정
args.save_results = "model_results_teamp3.json" # 모델 결과 저장 파일 이름 지정

# 데이터 분석을 위한 변수들
args.random_state = 21
args.results = [] # 모델 결과 저장을 위한 리스트

# 데이터 로딩

In [78]:
# 학습 데이터와 테스트 데이터 로딩
train_df = pd.read_csv(args.train_csv)
test_df = pd.read_csv(args.test_csv)

# 데이터 크기 확인
print("학습 데이터 크기:", train_df.shape)
print("테스트 데이터 크기:", test_df.shape)

# 학습 데이터 미리보기
print("\n학습 데이터 미리보기:")
display(train_df.head())

# 테스트 데이터 미리보기
print("\n테스트 데이터 미리보기:")
display(test_df.head())

학습 데이터 크기: (19158, 14)
테스트 데이터 크기: (2129, 13)

학습 데이터 미리보기:


,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
0,8949,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1.0
1,29725,city_40,0.776,Male,No relevent experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0.0
2,11561,city_21,0.624,NaN,No relevent experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0.0
3,33241,city_115,0.789,NaN,No relevent experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1.0
4,666,city_162,0.767,Male,Has relevent experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0.0



테스트 데이터 미리보기:


,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours
0,32403,city_41,0.827,Male,Has relevent experience,Full time course,Graduate,STEM,9,<10,NaN,1,21
1,9858,city_103,0.920,Female,Has relevent experience,no_enrollment,Graduate,STEM,5,NaN,Pvt Ltd,1,98
2,31806,city_21,0.624,Male,No relevent experience,no_enrollment,High School,NaN,<1,NaN,Pvt Ltd,never,15
3,27385,city_13,0.827,Male,Has relevent experience,no_enrollment,Masters,STEM,11,10/49,Pvt Ltd,1,39
4,27724,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,10000+,Pvt Ltd,>4,72


# 피처 드랍

In [79]:
# 드랍할 피처 목록
drop_features = ['enrollee_id', 'city', 'city_development_index', 'gender']

# 학습 데이터와 테스트 데이터에서 피처 드랍
train_df = train_df.drop(columns=drop_features)
test_df = test_df.drop(columns=drop_features)

print(f"\n드랍 후 학습 데이터 크기: {train_df.shape}")
print(f"드랍 후 테스트 데이터 크기: {test_df.shape}")


드랍 후 학습 데이터 크기: (19158, 10)
드랍 후 테스트 데이터 크기: (2129, 9)


# 결측치 처리 및 피처 엔지니어링

In [80]:
# 결측치 확인
print("학습 데이터 결측치:")
display(train_df.isnull().sum())

print("\n테스트 데이터 결측치:")
display(test_df.isnull().sum())

# 결측치 처리 (예시: 최빈값 또는 새로운 카테고리로 대체)
# 이 부분은 데이터 분석 결과에 따라 최적의 방법으로 수정될 수 있습니다.
for col in ['enrolled_university', 'education_level', 'major_discipline', 'experience', 'company_size', 'company_type', 'last_new_job']:
    if train_df[col].isnull().any():
        # 최빈값으로 대체 (간단한 예시)
        # mode_value = train_df[col].mode()[0]
        # train_df[col].fillna(mode_value, inplace=True)
        # if col in test_df.columns:
        #     test_df[col].fillna(mode_value, inplace=True)

        # 또는 'Unknown'과 같은 새로운 카테고리로 대체
        train_df[col].fillna('Unknown', inplace=True)
        if col in test_df.columns:
            test_df[col].fillna('Unknown', inplace=True)


print("\n결측치 처리 후 학습 데이터 결측치:")
display(train_df.isnull().sum())

print("\n결측치 처리 후 테스트 데이터 결측치:")
display(test_df.isnull().sum())

학습 데이터 결측치:


,0
relevent_experience,0
enrolled_university,386
education_level,460
major_discipline,2813
experience,65
company_size,5938
company_type,6140
last_new_job,423
training_hours,0
target,0



테스트 데이터 결측치:


,0
relevent_experience,0
enrolled_university,31
education_level,52
major_discipline,312
experience,5
company_size,622
company_type,634
last_new_job,40
training_hours,0



결측치 처리 후 학습 데이터 결측치:


/tmp/ipython-input-560478234.py:19: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_df[col].fillna('Unknown', inplace=True)
/tmp/ipython-input-560478234.py:21: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using

,0
relevent_experience,0
enrolled_university,0
education_level,0
major_discipline,0
experience,0
company_size,0
company_type,0
last_new_job,0
training_hours,0
target,0



결측치 처리 후 테스트 데이터 결측치:


,0
relevent_experience,0
enrolled_university,0
education_level,0
major_discipline,0
experience,0
company_size,0
company_type,0
last_new_job,0
training_hours,0


# 피처 엔지니어링: 새로운 피처 생성

In [81]:
# relevent_experience + major_discipline : rel_major_code
# 결합하여 새로운 범주형 피처 생성
train_df['rel_major_code'] = train_df['relevent_experience'].astype(str) + '_' + train_df['major_discipline'].astype(str)
test_df['rel_major_code'] = test_df['relevent_experience'].astype(str) + '_' + test_df['major_discipline'].astype(str)

# company_size + company_type + last_new_job : job_size_type_code3
# 결합하여 새로운 범주형 피처 생성
train_df['job_size_type_code3'] = train_df['company_size'].astype(str) + '_' + train_df['company_type'].astype(str) + '_' + train_df['last_new_job'].astype(str)
test_df['job_size_type_code3'] = test_df['company_size'].astype(str) + '_' + test_df['company_type'].astype(str) + '_' + test_df['last_new_job'].astype(str)



# 'experience' 피처를 수치형으로 변환하고 구간화
def categorize_experience(exp):
    if pd.isna(exp) or exp == 'Unknown':
        return -1 # 결측치는 -1로 표시
    elif exp == '<1':
        return 0
    elif exp == '>20':
        return 21 # 20년 초과는 21로 표시
    else:
        return int(exp)

train_df['experience_numeric'] = train_df['experience'].apply(categorize_experience)
test_df['experience_numeric'] = test_df['experience'].apply(categorize_experience)

# 경험 구간화 (예시: 0-1, 2-5, 6-10, 11-15, 16-20, >20)
bins = [-2, 1, 5, 10, 15, 20, np.inf] # -2는 결측치(-1) 포함
labels = [0, 1, 2, 3, 4, 5] # 각 구간에 대한 코드
# train_df['exp_bin_code'] = pd.cut(train_df['experience_numeric'], bins=bins, labels=labels, right=True, include_lowest=True).astype(int) # 정수 코드로 변환
# test_df['exp_bin_code'] = pd.cut(test_df['experience_numeric'], bins=bins, labels=labels, right=True, include_lowest=True).astype(int) # 정수 코드로 변환

# 구간화에 사용된 임시 컬럼 삭제
# train_df = train_df.drop(columns=['experience_numeric']) # training_per_year 계산을 위해 남겨둡니다.
# test_df = test_df.drop(columns=['experience_numeric']) # training_per_year 계산을 위해 남겨둡니다.


# rel_exp_score: 경력×관련경험 (float)
# relevant_experience를 수치형으로 변환 (Has relevent experience: 1, No relevent experience: 0)
train_df['relevent_experience_numeric'] = train_df['relevent_experience'].apply(lambda x: 1 if x == 'Has relevent experience' else 0)
test_df['relevent_experience_numeric'] = test_df['relevent_experience'].apply(lambda x: 1 if x == 'Has relevent experience' else 0)

# experience를 수치형으로 변환 (결측치는 0 또는 평균 등으로 대체 가능, 여기서는 0으로 대체)
# 위에서 experience_numeric을 만들었으므로 이를 활용
train_df['experience_for_score'] = train_df['experience_numeric'].apply(categorize_experience)
test_df['experience_for_score'] = test_df['experience_numeric'].apply(categorize_experience)

# 결측치(-1)를 0으로 대체하여 계산
train_df['experience_for_score'] = train_df['experience_for_score'].replace(-1, 0)
test_df['experience_for_score'] = test_df['experience_for_score'].replace(-1, 0)


# train_df['rel_exp_score'] = train_df['relevent_experience_numeric'] * train_df['experience_for_score']
# test_df['rel_exp_score'] = test_df['relevent_experience_numeric'] * test_df['experience_for_score']

# 사용된 임시 컬럼 삭제
# train_df = train_df.drop(columns=['relevent_experience_numeric', 'experience_for_score']) # training_rel/no_rel 계산을 위해 relevent_experience_numeric 남겨둡니다.
# test_df = test_df.drop(columns=['relevent_experience_numeric', 'experience_for_score']) # training_rel/no_rel 계산을 위해 relevent_experience_numeric 남겨둡니다.

# major_discipline 중 STEM 관련 여부 판단
train_df['is_stem'] = train_df['major_discipline'].apply(lambda x: 1 if x == 'STEM' else 0)
test_df['is_stem'] = test_df['major_discipline'].apply(lambda x: 1 if x == 'STEM' else 0)


# stem_related: STEM×관련경험 (0/1)
# train_df['stem_related'] = train_df['is_stem'] * train_df['relevent_experience_numeric'] # relevent_experience_numeric 사용
# test_df['stem_related'] = test_df['is_stem'] * test_df['relevent_experience_numeric'] # relevent_experience_numeric 사용

# 사용된 임시 컬럼 삭제
# train_df = train_df.drop(columns=['is_stem']) # training_stem/nonstem 계산을 위해 is_stem 남겨둡니다.
# test_df = test_df.drop(columns=['is_stem']) # training_stem/nonstem 계산을 위해 is_stem 남겨둡니다.


# --- 새로운 피처 추가 ---

# 1. training_per_year: training_hours / (experience_num+1)
# experience_numeric (결측치 -1 포함)를 사용하며, 0으로 나누는 것을 방지하기 위해 +1
train_df['training_per_year'] = train_df['training_hours'] / (train_df['experience_numeric'].replace(-1, 0) + 1) # 결측치(-1)는 0으로 대체하여 계산

# 2. training_rel / training_no_rel
# relevent_experience_numeric (Has relevent experience: 1, No relevent experience: 0) 사용
train_df['training_rel'] = train_df['training_hours'] * train_df['relevent_experience_numeric']
train_df['training_no_rel'] = train_df['training_hours'] * (1 - train_df['relevent_experience_numeric'])

test_df['training_per_year'] = test_df['training_hours'] / (test_df['experience_numeric'].replace(-1, 0) + 1) # 결측치(-1)는 0으로 대체하여 계산
test_df['training_rel'] = test_df['training_hours'] * test_df['relevent_experience_numeric']
test_df['training_no_rel'] = test_df['training_hours'] * (1 - test_df['relevent_experience_numeric'])


# 3. training_stem / training_nonstem
# is_stem (STEM: 1, Others: 0) 사용
train_df['training_stem'] = train_df['training_hours'] * train_df['is_stem']
test_df['training_stem'] = test_df['training_hours'] * test_df['is_stem']
train_df['training_nonstem'] = train_df['training_hours'] * (1 - train_df['is_stem'])
test_df['training_nonstem'] = test_df['training_hours'] * (1 - test_df['is_stem'])


# 4. training_expbin* 일괄 생성 (수치형)
# exp_bin_code를 원-핫 인코딩하고 각 더미와 training_hours 곱하기
# exp_bin_dummies_train = pd.get_dummies(train_df['exp_bin_code'], prefix='exp_bin', dtype=int)
# exp_bin_dummies_test = pd.get_dummies(test_df['exp_bin_code'], prefix='exp_bin', dtype=int)

# # training_hours와 곱하여 새로운 피처 생성
# for col in exp_bin_dummies_train.columns:
#     train_df[f'training_{col}'] = train_df['training_hours'] * exp_bin_dummies_train[col]
# for col in exp_bin_dummies_test.columns:
#      test_df[f'training_{col}'] = test_df['training_hours'] * exp_bin_dummies_test[col]

# 사용된 임시 컬럼들 정리
train_df = train_df.drop(columns=['experience_numeric', 'relevent_experience_numeric', 'experience_for_score', 'is_stem'])
test_df = test_df.drop(columns=['experience_numeric', 'relevent_experience_numeric', 'experience_for_score', 'is_stem'])


print("\n피처 엔지니어링 후 학습 데이터:")
display(train_df.head())

print("\n피처 엔지니어링 후 테스트 데이터:")
display(test_df.head())


피처 엔지니어링 후 학습 데이터:


,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target,rel_major_code,job_size_type_code3,training_per_year,training_rel,training_no_rel,training_stem,training_nonstem
0,Has relevent experience,no_enrollment,Graduate,STEM,>20,Unknown,Unknown,1,36,1.0,Has relevent experience_STEM,Unknown_Unknown_1,1.636364,36,0,36,0
1,No relevent experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0.0,No relevent experience_STEM,50-99_Pvt Ltd_>4,2.937500,0,47,47,0
2,No relevent experience,Full time course,Graduate,STEM,5,Unknown,Unknown,never,83,0.0,No relevent experience_STEM,Unknown_Unknown_never,13.833333,0,83,83,0
3,No relevent experience,Unknown,Graduate,Business Degree,<1,Unknown,Pvt Ltd,never,52,1.0,No relevent experience_Business Degree,Unknown_Pvt Ltd_never,52.000000,0,52,0,52
4,Has relevent experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0.0,Has relevent experience_STEM,50-99_Funded Startup_4,0.363636,8,0,8,0



피처 엔지니어링 후 테스트 데이터:


,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,rel_major_code,job_size_type_code3,training_per_year,training_rel,training_no_rel,training_stem,training_nonstem
0,Has relevent experience,Full time course,Graduate,STEM,9,<10,Unknown,1,21,Has relevent experience_STEM,<10_Unknown_1,2.100000,21,0,21,0
1,Has relevent experience,no_enrollment,Graduate,STEM,5,Unknown,Pvt Ltd,1,98,Has relevent experience_STEM,Unknown_Pvt Ltd_1,16.333333,98,0,98,0
2,No relevent experience,no_enrollment,High School,Unknown,<1,Unknown,Pvt Ltd,never,15,No relevent experience_Unknown,Unknown_Pvt Ltd_never,15.000000,0,15,0,15
3,Has relevent experience,no_enrollment,Masters,STEM,11,10/49,Pvt Ltd,1,39,Has relevent experience_STEM,10/49_Pvt Ltd_1,3.250000,39,0,39,0
4,Has relevent experience,no_enrollment,Graduate,STEM,>20,10000+,Pvt Ltd,>4,72,Has relevent experience_STEM,10000+_Pvt Ltd_>4,3.272727,72,0,72,0


# 피처 및 타겟 변수 분리

In [82]:
# 타겟 변수 분리
X = train_df.drop('target', axis=1)
y = train_df['target']

# 테스트 데이터는 타겟 변수가 없으므로 X_test로 지정
X_test = test_df.copy()

print("학습 데이터 피처 (X) 크기:", X.shape)
print("학습 데이터 타겟 (y) 크기:", y.shape)
print("테스트 데이터 피처 (X_test) 크기:", X_test.shape)

학습 데이터 피처 (X) 크기: (19158, 16)
학습 데이터 타겟 (y) 크기: (19158,)
테스트 데이터 피처 (X_test) 크기: (2129, 16)


# 피처 인코딩

In [83]:
# 범주형 및 순서형 피처 정의 (생성된 새로운 피처 포함)
categorical_features = ['relevent_experience', 'enrolled_university', 'major_discipline',
                      'company_size', 'company_type', 'last_new_job',
                      'rel_major_code', 'job_size_type_code3'] # stem_related 제거

# 순서형 피처 정의 (교육 수준 및 경험 구간)
# education_level: Primary School < High School < Graduate < Masters < PhD
# experience: <1 < 1 ... < 20 < >20
# exp_bin_code: 0 < 1 < 2 < 3 < 4 < 5
ordinal_features = ['education_level', 'experience'] # exp_bin_code 제거

# 순서 정의
education_level_order = ['Primary School', 'High School', 'Graduate', 'Masters', 'PhD', 'Unknown']
experience_order = ['<1', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '>20', 'Unknown']
# exp_bin_code_order = [0, 1, 2, 3, 4, 5] # 정수형이므로 순서대로

# 원-핫 인코딩 및 순서형 인코딩을 위한 ColumnTransformer 생성
# Unknown 카테고리를 포함하여 처리
preprocessor = ColumnTransformer(
    transformers=[
        ('onehot', OneHotEncoder(handle_unknown='ignore'), categorical_features), # 알 수 없는 범주는 무시
        ('ordinal_edu', OrdinalEncoder(categories=[education_level_order], handle_unknown='use_encoded_value', unknown_value=-1), ['education_level']), # 알 수 없는 범주는 -1로 인코딩
        ('ordinal_exp', OrdinalEncoder(categories=[experience_order], handle_unknown='use_encoded_value', unknown_value=-1), ['experience']), # 알 수 없는 범주는 -1로 인코딩
        # ('ordinal_exp_bin', OrdinalEncoder(categories=[exp_bin_code_order], handle_unknown='use_encoded_value', unknown_value=-1), ['exp_bin_code']) # exp_bin_code 제거
    ],
    remainder='passthrough' # 나머지 수치형 피처는 그대로 통과
)

# 데이터에 전처리 적용
X_processed = preprocessor.fit_transform(X)
X_test_processed = preprocessor.transform(X_test) # 테스트 데이터는 학습 데이터의 인코더를 사용

print("\n전처리 후 학습 데이터 크기:", X_processed.shape)
print("전처리 후 테스트 데이터 크기:", X_test_processed.shape)


전처리 후 학습 데이터 크기: (19158, 409)
전처리 후 테스트 데이터 크기: (2129, 409)


# 모델 학습: Decision Tree, 튜닝,교차

In [87]:
# Decision Tree 모델 정의
# 데이터 불균형 처리를 위해 class_weight='balanced' 설정 추가
dt_model = DecisionTreeClassifier(random_state=args.random_state, class_weight='balanced')

# 탐색할 하이퍼파라미터 그리드 정의 (예시, 필요에 따라 수정 가능)
# Decision Tree 모델에 맞는 파라미터들을 탐색합니다.
param_grid = {
    'max_depth': [None, 5, 10, 15, 20], # 트리의 최대 깊이
    'min_samples_split': [2, 5, 10, 20], # 노드를 분할하기 위한 최소 샘플 수
    'min_samples_leaf': [1, 2, 4, 8], # 리프 노드가 되기 위한 최소 샘플 수
    'criterion': ['gini', 'entropy'] # 분할 품질 측정 기준
    # class_weight를 튜닝 파라미터로 포함시킬 수도 있습니다: 'class_weight': [None, 'balanced']
}

# GridSearchCV를 사용한 하이퍼파라미터 튜닝
# 최적의 파라미터 조합을 찾기 위해 교차 검증을 수행합니다.
# ROC AUC 점수를 기준으로 평가합니다.
grid_search = GridSearchCV(dt_model, param_grid, cv=5, scoring='roc_auc', n_jobs=-1)

print("하이퍼파라미터 튜닝 시작...")
grid_search.fit(X_processed, y) # 학습 데이터로 그리드 서치 수행
print("하이퍼파라미터 튜닝 완료.")

# 최적의 파라미터와 성능 확인
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("\n최적 하이퍼파라미터:", best_params)
print("최적 ROC AUC 점수 (GridSearchCV 교차 검증):", best_score) # 출력 메시지 수정

# 최적의 모델 선택
best_dt_model = grid_search.best_estimator_

# 최적 모델의 다양한 평가지표 계산 (교차 검증)
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

scoring = {
    'accuracy': make_scorer(accuracy_score),
    'precision': make_scorer(precision_score),
    'recall': make_scorer(recall_score),
    'f1_score': make_scorer(f1_score),
    # 'roc_auc': make_scorer(roc_auc_score, needs_proba=True) # AUC는 확률이 필요 - nan 문제로 인해 별도 계산
}

print("\n최적 모델의 교차 검증 평가지표:")
cv_results = cross_validate(best_dt_model, X_processed, y, cv=5, scoring=scoring, n_jobs=-1)

# 결과 출력
print(f"  정확도 (Accuracy): {cv_results['test_accuracy'].mean():.4f}")
print(f"  정밀도 (Precision): {cv_results['test_precision'].mean():.4f}")
print(f"  재현율 (Recall): {cv_results['test_recall'].mean():.4f}")
print(f"  F1-스코어 (F1 Score): {cv_results['test_f1_score'].mean():.4f}")

# 최적 모델을 전체 학습 데이터에 대해 다시 학습 (GridSearchCV 결과 사용)
# best_dt_model 이미 GridSearchCV에서 학습된 최적 모델입니다.

# 전체 학습 데이터에 대한 예측 확률 계산 및 ROC AUC 계산
y_pred_proba_train = best_dt_model.predict_proba(X_processed)[:, 1]
overall_roc_auc = roc_auc_score(y, y_pred_proba_train)

print(f"  전체 학습 데이터에 대한 AUC (ROC AUC): {overall_roc_auc:.4f}") # 별도 계산된 AUC 출력

하이퍼파라미터 튜닝 시작...
하이퍼파라미터 튜닝 완료.

최적 하이퍼파라미터: {'criterion': 'entropy', 'max_depth': 5, 'min_samples_leaf': 1, 'min_samples_split': 2}
최적 ROC AUC 점수 (GridSearchCV 교차 검증): 0.7155720834455603

최적 모델의 교차 검증 평가지표:
  정확도 (Accuracy): 0.6643
  정밀도 (Precision): 0.4019
  재현율 (Recall): 0.6569
  F1-스코어 (F1 Score): 0.4934
  전체 학습 데이터에 대한 AUC (ROC AUC): 0.7280


# 예측 및 결과 생성

In [85]:
# 테스트 데이터에 대한 예측 확률 생성
# Decision Tree는 predict_proba를 통해 클래스별 확률을 반환합니다.
# 여기서는 타겟 클래스 1 (입사)에 대한 확률을 사용합니다.
test_predictions = best_dt_model.predict_proba(X_test_processed)[:, 1]

# 제출 파일 생성을 위해 원본 테스트 데이터 로드 (enrollee_id를 가져오기 위함)
original_test_df = pd.read_csv(args.test_csv)

# 제출 파일 형식에 맞게 DataFrame 생성
submission_df = pd.DataFrame({'enrollee_id': original_test_df['enrollee_id'], 'target': test_predictions})

# 제출 파일 저장
submission_df.to_csv(args.submission_csv, index=False)

print(f"제출 파일이 '{args.submission_csv}'로 저장되었습니다.")

제출 파일이 'submission_teamp3.csv'로 저장되었습니다.


# 결과 저장

In [86]:
# 모델 결과 저장 (예시)
# 저장할 결과 정보 딕셔너리 생성
results_entry = {
    'model': 'Decision Tree (Tuned)',
    'features': 'Selected and Engineered', # 사용된 피처에 대한 설명
    'roc_auc_score': best_score, # 교차 검증 최고 ROC AUC 점수
    'best_params': best_params # 최적 하이퍼파라미터
}

# 기존 결과 파일이 있다면 로드
try:
    with open(args.save_results, 'r', encoding='utf-8') as f:
        args.results = json.load(f)
except (FileNotFoundError, json.JSONDecodeError):
    args.results = [] # 파일이 없거나 형식이 잘못되었으면 빈 리스트로 초기화

# 새로운 결과 추가
args.results.append(results_entry)

# 결과를 JSON 파일로 저장 (한글 인코딩 설정)
with open(args.save_results, 'w', encoding='utf-8') as f:
    json.dump(args.results, f, indent=4, ensure_ascii=False)

print(f"모델 결과가 '{args.save_results}'에 저장되었습니다.")
display(args.results)

모델 결과가 'model_results_teamp3.json'에 저장되었습니다.


[{'model': 'Decision Tree (Tuned)',
  'features': 'Selected and Engineered',
  'roc_auc_score': 0.7157563295704206,
  'best_params': {'criterion': 'gini',
   'max_depth': 5,
   'min_samples_leaf': 2,
   'min_samples_split': 2}},
 {'model': 'Decision Tree (Tuned)',
  'features': 'Selected and Engineered',
  'roc_auc_score': 0.7157563295704206,
  'best_params': {'criterion': 'gini',
   'max_depth': 5,
   'min_samples_leaf': 2,
   'min_samples_split': 2}},
 {'model': 'Decision Tree (Tuned)',
  'features': 'Selected and Engineered',
  'roc_auc_score': 0.7142685114764346,
  'best_params': {'criterion': 'gini',
   'max_depth': 5,
   'min_samples_leaf': 2,
   'min_samples_split': 10}},
 {'model': 'Decision Tree (Tuned)',
  'features': 'Selected and Engineered',
  'roc_auc_score': np.float64(0.7155720834455603),
  'best_params': {'criterion': 'entropy',
   'max_depth': 5,
   'min_samples_leaf': 1,
   'min_samples_split': 2}}]